In [1]:
import pandas as pd
import numpy as np

I am going to do transformations for both the cleaned data with missing values and the imputed data

In [2]:
df = pd.read_csv('../Data/df_cleaned2.csv')
df_i = pd.read_csv('../Data/imputed.csv')

In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df_i.drop('Unnamed: 0',axis=1,inplace=True)
df.columns

Index(['LBXTC', 'RIAGENDR', 'RIDRETH3', 'RIDAGEYR', 'BMXWT', 'BMXHT', 'BMXBMI',
       'BPXPLS', 'BPXPULS', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA', 'DR1_300',
       'ALQ120Q', 'ALQ120U', 'ALQ130', 'BPQ020', 'BPQ050A', 'BPQ080',
       'BPQ100D', 'DIQ010', 'MCQ170M', 'SMQ681', 'PAQ610', 'PAD615', 'PAQ625',
       'PAD630', 'PAQ655', 'PAD660', 'PAQ670', 'PAD675', 'Systolic',
       'Diastolic', 'outlier'],
      dtype='object')

# Engineering physicial activity data for work and recreation

According to the codebook, the amount of physical activity comes with a MET score for weighing.  Vigorous activity gets a score of 8 while moderate activity gets a score of 4.  
What I plan to do is calculate the daily average MET amount of minutes for physical activity for both work and rescreational activity by first multiplying the vigours time by 8, moderate amounts by 4, and adding them together.  Then I divide by 7 to get a daily average.

In [7]:
df_i.columns

Index(['LBXTC', 'RIAGENDR', 'RIDRETH3', 'RIDAGEYR', 'BMXWT', 'BMXHT', 'BMXBMI',
       'BPXPLS', 'BPXPULS', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA', 'DR1_300',
       'ALQ120Q', 'ALQ120U', 'ALQ130', 'BPQ020', 'BPQ050A', 'BPQ080',
       'BPQ100D', 'DIQ010', 'MCQ170M', 'SMQ681', 'PAQ610', 'PAD615', 'PAQ625',
       'PAD630', 'PAQ655', 'PAD660', 'PAQ670', 'PAD675', 'Systolic',
       'Diastolic', 'outlier', 'BMXWT_i', 'BMXHT_i', 'BMXBMI_i', 'BPXPLS_i',
       'BPXPULS_i', 'DR1_300_i', 'ALQ120Q_i', 'ALQ120U_i', 'ALQ130_i',
       'BPQ020_i', 'BPQ050A_i', 'BPQ080_i', 'BPQ100D_i', 'DIQ010_i',
       'MCQ170M_i', 'SMQ681_i', 'PAQ610_i', 'PAD615_i', 'PAQ625_i', 'PAD630_i',
       'PAQ655_i', 'PAD660_i', 'PAQ670_i', 'PAD675_i', 'Systolic_i',
       'Diastolic_i'],
      dtype='object')

In [4]:
df['MET_work'] = 8*df.PAQ610*df.PAD615/7 + 4*df.PAQ625*df.PAD630/7
df['MET_rec'] = 8*df.PAQ655*df.PAD660/7 + 4*df.PAQ670*df.PAD675/7
df_i['MET_work'] = 8*df_i.PAQ610*df_i.PAD615/7 + 4*df_i.PAQ625*df_i.PAD630/7
df_i['MET_rec'] = 8*df_i.PAQ655*df_i.PAD660/7 + 4*df_i.PAQ670*df_i.PAD675/7
# here I need to add a met_rec i values
if sum(df.PAQ610.isna()) + sum(df.PAD615.isna()) + sum(df.PAQ625.isna()) + sum(df.PAD630.isna()) > 0:
    df_i['MET_work_i']=0
    df_i.loc[df.PAQ610!=df_i.PAQ610,'MET_work_i']=1
    # need to do this for the rest

# Weighed nutrition values



In [5]:
df.DR1_300.unique()

array([ 2.,  1.,  3., nan])

In [6]:
df.DR1_300.value_counts()

2.0    3298
3.0     916
1.0     410
Name: DR1_300, dtype: int64

With DR1_300 1 means more than usual was consumed, 2 means usual amount was consumed and 3 means much less than usual was consumed.  
For those where more than usuals was consumed.  I plan to create transformed nutrient information by multiplying the values by 0.5.  When less than usual was consumed, I plan to multiply the variables by 2.

In [7]:
powers = df.DR1_300 - 2
powers_i = df_i.DR1_300 - 2
nutrtition_variables = ['DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA']

In [8]:
transformed_variables = []
for var in nutrtition_variables:
    name = "{}_t".format(var)
    transformed_variables.append(name)
    df[name] = df[var]*2**powers
    df_i[name] = df_i[var]*2**powers_i

In [9]:
#just checking to make sure it worked
df[nutrtition_variables[:3]+['DR1_300']+transformed_variables[:3]].head()

,DR1TKCAL,DR1TPROT,DR1TCARB,DR1_300,DR1TKCAL_t,DR1TPROT_t,DR1TCARB_t
0,2964.0,62.36,356.85,2.0,2964.0,62.36,356.850
1,2482.0,116.50,304.53,1.0,1241.0,58.25,152.265
2,604.0,19.30,90.30,3.0,1208.0,38.60,180.600
3,1304.0,79.02,153.43,2.0,1304.0,79.02,153.430
4,2338.0,102.10,282.58,2.0,2338.0,102.10,282.580


In [10]:
df.columns

Index(['LBXTC', 'RIAGENDR', 'RIDRETH3', 'RIDAGEYR', 'BMXWT', 'BMXHT', 'BMXBMI',
       'BPXPLS', 'BPXPULS', 'DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA', 'DR1_300',
       'ALQ120Q', 'ALQ120U', 'ALQ130', 'BPQ020', 'BPQ050A', 'BPQ080',
       'BPQ100D', 'DIQ010', 'MCQ170M', 'SMQ681', 'PAQ610', 'PAD615', 'PAQ625',
       'PAD630', 'PAQ655', 'PAD660', 'PAQ670', 'PAD675', 'Systolic',
       'Diastolic', 'outlier', 'MET_work', 'MET_rec', 'DR1TKCAL_t',
       'DR1TPROT_t', 'DR1TCARB_t', 'DR1TSUGR_t', 'DR1TFIBE_t', 'DR1TTFAT_t',
       'DR1TSFAT_t', 'DR1TMFAT_t', 'DR1TPFAT_t', 'DR1TCHOL_t', 'DR1TSODI_t',
       'DR1TVD_t', 'DR1TCALC_t', 'DR1TIRON_t', 'DR1TPOTA_t'],
      dtype='object')

# Average daily alcohol consumption
ALQ120Q asks how often do you drink alcohol in the past year
ALQ120U asks the unit where
1 means week
2 means month
3 means year

I am going to replace week with 52, month with 12, year with 1 then multiiply these balues by ALQ120Q and the by ALQ130 (how many drinks on days you drink) then divide by 365 to get the daily average

In [11]:
df['avgALC'] = df.ALQ120Q*df.ALQ120U.replace({1:52,2:12,3:1})*df.ALQ130/365
df_i['avgALC'] = df_i.ALQ120Q*df_i.ALQ120U.replace({1:52,2:12,3:1})*df_i.ALQ130/365

## Log transform the dependant variable

As I said in my eda, I may try with log of LBXTC

In [12]:
df['log_LBXTC']=np.log(df.LBXTC)
df_i['log_LBXTC']=np.log(df_i.LBXTC)


In [13]:
df.to_csv('../Data/df_transformed.csv')

In [14]:
df_i.to_csv('../Data/dfi_transformed.csv')